# Imports

In [0]:
# Import modules
from pyspark.sql import functions as F
import pyspark.sql.types as DataTypes

from dotenv import load_dotenv
import os

# Data Import and Cleaning

In [0]:
# Set file path to data
filepath = os.getenv('BUCKET_PATH')

# Create data frame
data_clean = spark.read.json(filepath)

# Create working data frame and remove top level
data_clean = data_clean.select('data.*')

display(data_clean.limit(5))
data_clean.printSchema()

appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000","List(true, true, true)",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,"List(266, 1191, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5426, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 227, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2784, null, null, null, null, null, null, null, null, null, null, null, null, 1607, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4831, null, null, null, null, null, null, null, null, null, 1707, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 632, null, null, null, null, null, null, null, null, null, null, null, 3392, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 131, null, null, 769, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 881, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 289, null, null, null, 3353, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 614, null, null, null, null, null, null, 304, null, null, null, 1344, null, null, 1864, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1192)",game,
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/1000000/header.jpg?t=1655723048,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","List(false, false, true)",27,999,PsychoFlux Entertainment,2021/05/14,0,"ASCENXION is a 2D shoot 'em up game where you explore the field to progress. P

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- platforms: struct (nullable = true)
 |    |-- linux: boolean (nullable = true)
 |    |-- mac: boolean (nullable = true)
 |    |-- windows: boolean (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- tags: struct (nullable = true)
 |    |-- 1980s: lon

In [0]:
# Transform and clean data

# Separate platforms into columns
# Calculate rating based on reviews
# Convert release date to datetime
# Find lower bound of owners from interval
# Cast columns to correct types
# Remove useless columns
data = data_clean.withColumn('linux', F.col('platforms.linux')) \
    .withColumn('mac', F.col('platforms.mac')) \
    .withColumn('windows', F.col('platforms.windows')) \
    .withColumn('rating', F.col('positive') / (F.col('positive') + F.col('negative'))) \
    .withColumn('release_date', F.to_date(F.unix_timestamp(F.col('release_date'), 'yyyy/M/d').cast("timestamp"))) \
    .withColumn('price', F.col('price').cast(DataTypes.FloatType())) \
    .withColumn('initialprice', F.col('initialprice').cast(DataTypes.FloatType())) \
    .withColumn('discount', F.col('discount').cast(DataTypes.FloatType())) \
    .drop('platforms') \
    .drop('tags') \
    .drop('categories') \
    .drop('website') \
    .drop('short_description') \
    .drop('header_image')

# Extract owner uppper and lower bounds
# Calculate revenue
data = data.withColumn('owners', F.split('owners', ' ')) \
    .withColumn('lower_owners', F.col('owners')[0]) \
    .withColumn('upper_owners', F.col('owners')[2]) \
    .withColumn('lower_owners', F.regexp_replace(F.col('lower_owners'), ',', '')) \
    .withColumn('upper_owners', F.regexp_replace(F.col('upper_owners'), ',', '')) \
    .withColumn('lower_owners', F.col('lower_owners').cast(DataTypes.IntegerType())) \
    .withColumn('upper_owners', F.col('upper_owners').cast(DataTypes.IntegerType())) \
    .withColumn('owners', (F.col('lower_owners') + F.col('upper_owners')) / 2) \
    .withColumn('revenue', F.col('owners') * F.col('price')) \
    .drop('lower_owners') \
    .drop('upper_owners')

# Explode genre
data = data.withColumn('genre', F.split(data['genre'], ','))
data = data.withColumn('genre', F.explode('genre'))

# Explode language
data = data.withColumn('languages', F.split(data['languages'], ','))
data = data.withColumn('languages', F.explode('languages'))

In [0]:
display(data.limit(5))

appid,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age,type,linux,mac,windows,rating,revenue
10,13990,Valve,0.0,Action,999.0,English,Counter-Strike,5199,1.5E7,201215,999.0,Valve,2000-11-01,0,game,true,true,true,0.9748127549487923,1.4985E10
10,13990,Valve,0.0,Action,999.0,French,Counter-Strike,5199,1.5E7,201215,999.0,Valve,2000-11-01,0,game,true,true,true,0.9748127549487923,1.4985E10
10,13990,Valve,0.0,Action,999.0,German,Counter-Strike,5199,1.5E7,201215,999.0,Valve,2000-11-01,0,game,true,true,true,0.9748127549487923,1.4985E10
10,13990,Valve,0.0,Action,999.0,Italian,Counter-Strike,5199,1.5E7,201215,999.0,Valve,2000-11-01,0,game,true,true,true,0.9748127549487923,1.4985E10
10,13990,Valve,0.0,Action,999.0,Spanish - Spain,Counter-Strike,5199,1.5E7,201215,999.0,Valve,2000-11-01,0,game,true,true,true,0.9748127549487923,1.4985E10


In [0]:
data.count()

Out[69]: 579641

# Macro Analysis

#### Which publisher has the most games on Steam?

In [0]:
# Count of games by publisher
display(
    data.groupBy('publisher').count().orderBy('count', ascending=False).limit(5)
)

publisher,count
Hede,8121
Dnovel,2543
Blender Games,2274
Ubisoft,2135
Daedalic Entertainment,1920


Interestingly enough, the publisher with the most titles (by a significant amount) is a known "scam" publisher. We also see Blender Games which publishes large amounts of very inexpensive "junk" games. Other than that, we unsurprisingly see Ubisoft, a large publisher, and Deadalic Entertainment, a renowned indie game publisher.

#### What are the best rated games?

In [0]:
# Find games with most postive reviews and show their ratings too
display(
    data.groupBy('name').max('positive', 'rating').orderBy('max(positive)', ascending=False).limit(30)
)

name,max(positive),max(rating)
Counter-Strike: Global Offensive,5943345,0.8830547135268165
Dota 2,1534895,0.828414231133127
Grand Theft Auto V,1229265,0.852091714934523
PUBG: BATTLEGROUNDS,1185361,0.5661084992616564
Terraria,1014711,0.978420408623737
Tom Clancy's Rainbow Six Siege,942910,0.8681157512219688
Garry's Mod,861240,0.9663412017889722
Team Fortress 2,846407,0.9364670347299824
Rust,732513,0.8672210468740936
Left 4 Dead 2,643836,0.9745286560187932


Overall, we see that the most popular games by amount of positive reviews also have very high ratings (generally above or around 85%, which corresponds to 'very positive' on Steam). We do see an outlier: PUBG: Battlegrounds. Despite the game's enormous popularity, it has poor overall ratings due to it's shoddy performance, rough launch and lackluster updates. However, its status as one of (if not the) first battle royale games has cemented it as one of the most played (and thus rated) games on steam.

#### Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

In [0]:
display(
    data.groupBy(F.year('release_date')).count().orderBy('count', ascending=False).limit(5),
    data.groupBy(F.year('release_date')).count().orderBy('count', ascending=True).limit(5)
)

year(release_date),count
2021,100655
2020,94501
2022,88114
2019,74992
2018,74657


Unsurprisingly, we see that the amount of releases by year has trended with the growth of the video game industry: many more games have been releaased recently. That being said, it doesn't follow a strict increasing pattern, we see a drop in 2022 that we didn't expect. Let's take a look at this data in chronological order:

In [0]:
display(
    data.groupBy(F.year('release_date')).count().orderBy(F.year('release_date'), ascending=False).limit(10)
)

year(release_date),count
2022,88114
2021,100655
2020,94501
2019,74992
2018,74657
2017,54082
2016,36330
2015,23607
2014,14203
2013,5807


It seems like the only year that falls out of our pattern of steady increase is 2022, which could be explained by our dataset possibly being released before the end of said year. Covid, which began in early 2022, seems to have only have had a positive impact on the amount of releases. This could be explained by the boom of small development teams that came together during quarantine to create their own indie titles.

#### How are the prices distributed? Are there any games with a discount?

In [0]:
display(
    data.groupBy('price').count().orderBy('price', ascending=False).limit(5),
    data.groupBy('price').count().orderBy('count', ascending=False).limit(5)
)

price,count
99900.0,1
29990.0,1
26999.0,4
24900.0,4
19999.0,89


It looks like we have an enormous amount of games with very high prices, lets try to dive a bit deeper into these results.

In [0]:
data_price = data.filter(
    (F.col('name') == 'Grand Theft Auto V') | \
    (F.col('name') == 'Terraria') | \
    (F.col('name') == 'Monster Hunter: World') | \
    (F.col('name') == 'Left 4 Dead 2') | \
    (F.col('name') == "Garry's Mod") | \
    (F.col('name') == 'Rust') | \
    (F.col('name') == 'ARK: Survival Evolved')) \
        .select(['name', 'developer', 'price']).groupBy('name').max()

display(data_price)

name,max(price)
Left 4 Dead 2,999.0
Terraria,999.0
Rust,3999.0
Grand Theft Auto V,2998.0
Garry's Mod,999.0
Monster Hunter: World,2999.0
ARK: Survival Evolved,1999.0


Despite the prices initially seeming incorrect, we can see that the prices do actually seem correct relative to each other when looking at some popular games. Let's add the real prices and find the differences.

In [0]:
actual_prices = {
    "Left 4 Dead 2": 9.99,
    "Terraria": 9.99,
    "Rust": 39.99,
    "Grand Theft Auto V": 29.99,
    "Garry's Mod": 9.99,
    "Monster Hunter: World": 59.99,
    "ARK: Survival Evolved": 39.99
}

data_price = data_price.join(spark.createDataFrame(actual_prices.items(), ["name", "actual_price_usd"]), "name")
data_price = data_price.withColumn('difference', F.col("max(price)") / F.col("actual_price_usd"))
display(data_price)

name,max(price),actual_price_usd,difference
Left 4 Dead 2,999.0,9.99,100.0
Terraria,999.0,9.99,100.0
Rust,3999.0,39.99,100.0
Grand Theft Auto V,2998.0,29.99,99.96665555185062
Garry's Mod,999.0,9.99,100.0
Monster Hunter: World,2999.0,59.99,49.991665277546254
ARK: Survival Evolved,1999.0,39.99,49.98749687421855


It looks like the prices in the dataset have been multiplied by 100, so we will simply divide all price values by 100.

In [0]:
data = data.withColumn('price', F.col('price') / 100) \
    .withColumn('initialprice', F.col('initialprice') / 100) \
    .withColumn('revenue', F.col('revenue') / 100)

In [0]:
display(data.select('price').describe())
data.approxQuantile('price', [0.25, 0.5, 0.75], 0)

summary,price
count,579641
mean,9.006299796013808
stddev,11.138714939416925
min,0.0
max,999.0


Out[78]: [1.59, 4.99, 12.99]

The mean price of a game is around $9 USD, with a standard deviation of $11 USD. 
We remark that our median is actually much lower than our mean, at $4.99 indicating that our prices are quite significantly positively skewed (there are many high price outliers).

The minimum and maximum sit at the extremes of free and $999.

In [0]:
display(data.filter(F.col('price') == 999).select(['name', 'genre', 'developer', 'price']))

name,genre,developer,price
Ascent Free-Roaming VR Experience,Action,Fury Games,999.0


Out of curiousity, we find that there is a single game on Steam priced at $999: 'Ascent Free-Roaming VR Experience'

In [0]:
data_discount = data.groupBy('name').max().filter(F.col('max(discount)') > 0)

In [0]:
print(
    'Amount of games with a discount: ',
   round(data_discount.count() / data.count() * 100, 3),
   '%'
)

print(
    'Average discount for games that do have a discount: ',
    round(data_discount.select(F.mean('max(discount)')).collect()[0][0], 2),
    '%'
)

Amount of games with a discount:  0.434 %
Average discount for games that do have a discount:  57.59 %



The average discount for games that *do* have a discount is 60%, however, very few games are discounted on Steam overall: perhaps this is due to the many "junk" games on the platform that are rarely updated or played. Let's try to filter these out by only looking at games with more than 1,000 reviews.

In [0]:

print(
    'Average owners of discounted games:',  
    round(data_discount.filter((F.col('max(positive)') + F.col('max(negative)')) > 1000).select(F.mean('max(owners)')).collect()[0][0], 0),
    '\n',
    'Average owners of undiscounted games:',
    round(data.groupBy('name').max().filter((F.col('max(positive)') + F.col('max(negative)')) > 1000).select(F.mean('max(owners)')).collect()[0][0], 0))

Average owners of discounted games: 883099.0 
 Average owners of undiscounted games: 1143049.0




When filtering out games with a low amount of total reviews, we find that undiscounted games actually have a higher amount of average owners. Perhaps games discount themselves hoping to get more users, indicating they had few in the first place.

#### What are the most represented languages?

In [0]:
display(
    data.groupBy('languages').count().orderBy('count', ascending=False).limit(10),
    data.groupBy('languages').count().orderBy('count', ascending=True).limit(5)
)

languages,count
English,154137
German,38220
Simplified Chinese,36910
Russian,36872
French,36590
Spanish - Spain,33859
Japanese,28191
Italian,25104
Portuguese - Brazil,19650
Korean,19022


Unsurprisingly, we see English as the most common language for steam games with 4 times more games than any other language. Closely following, we have most major european lanugages (German, Russian, French, Spanish, Italian) alongside a few asian languages such as Simplified/Traditional Mandarin and Korean.

Our 5 least popular languages contain 4 'languages' that are actually variations on our most common languages. With some effort cleaning this data, we could get a better representation of which languages are the least supported.

In [0]:
data.filter(F.col('languages') != 'English').groupBy('name').max().count() / data.count() * 100

Out[84]: 4.5697250539558105

It looks like we only have 4.5% of all games that *do not* have English as a language.

#### Are there any games prohibited from children under 16/18?

In [0]:
print(data.filter(F.col('required_age') > 0).count() / data.count() * 100)
print(data.filter(F.col('required_age') >= 18).count() / data.count() * 100)

1.3882730862723651
0.4347518550275084


Only 1.4% of games feature an age rating above 0, meaning a vast majority of games on Steam are considered suitable for everyone. We could anticipate that this is done so that developers can maximise the accessibility of their games, although this doesn't take into account the possibility that some games are simply 'unrated' due to a lack of means. We will explore this in our next step.

An even more impressively low 0.4% of games are rated over 18, showing just how few games are targeted at mature audiences.

In [0]:
print(
    data.filter(F.col('required_age') > 0).groupBy('name').max().select(F.median('max(owners)')).show(),
    data.groupBy('name').max().select(F.median('max(owners)')).show(),
    data.filter(F.col('required_age') > 0).groupBy('name').max().select(F.median('max(revenue)')).show(),
    data.groupBy('name').max().select(F.median('max(revenue)')).show()
)

+-------------------+
|median(max(owners))|
+-------------------+
|            75000.0|
+-------------------+

+-------------------+
|median(max(owners))|
+-------------------+
|            10000.0|
+-------------------+

+--------------------+
|median(max(revenue))|
+--------------------+
|            452275.0|
+--------------------+

+--------------------+
|median(max(revenue))|
+--------------------+
|             59900.0|
+--------------------+

None None None None



When we look at purchase and revenue amounts, we find some interesting insights. It seems as though games that are rated actually have significantly higher player bases and revenue amounts. This actually points us to our previous remark, the possibility that games are unrated due to a lack of means or effort.

If we look at the games we used to analyze our prices, we see that most of those games did actually feature an age rating. Maybe we can suppose that games that have the means (and necessity) to receive a proper age rating score are also games produced by the largest studios

# Genre Analysis

#### What are the most represented genres?

In [0]:
display(
    data.groupBy('genre').count().orderBy('count', ascending=False).limit(10)
)

genre,count
Indie,124887
Action,87070
Casual,47249
Adventure,41376
Casual,39712
Adventure,39032
Strategy,39010
Simulation,37902
RPG,28607
Early Access,16480


Indie games are the most popular genre, which is expected given that they are also the games that generally require the lowest effort and development team size. Apart from that, the top genres are those we expect to be most common.

#### Are there any genres that have a better positive/negative review ratio?

In [0]:
display(
    data.groupBy('genre').agg(F.mean('rating').alias('average_rating'), F.count('rating').alias('games_in_genre')).orderBy('average_rating', ascending=False).limit(10),
    data.groupBy('genre').agg(F.mean('rating').alias('average_rating'), F.count('rating').alias('games_in_genre')).orderBy('average_rating', ascending=True).limit(10)
)

genre,average_rating,games_in_genre
Game Development,0.875543159820522,45
Early Access,0.7843566496034051,54
Adventure,0.7780953752061316,38944
Casual,0.7706988605203206,39489
RPG,0.7654013017744452,2616
Indie,0.7641087007044408,124415
Strategy,0.7551537898233701,2772
Casual,0.7545438234542035,47082
Indie,0.747609130221416,16237
Action,0.7460070916159153,86864


Once more, we see our most common genres as those with the highest average ratings. They all hover around the same score of 75%, which roughly corresponds to the average rating of games on steam in general (see next code block). We also note 2 outliers with exceptionally high average ratings.

In [0]:
display(data.groupBy('name').agg(F.max('rating').alias('rating')).select(F.mean('rating')))

avg(rating)
0.7375417323235289


What is more interesting to note, is that all of the worst rated genres are quite niche, under a few hundred games whereas the most popular genres had 10s of thousands of games.

#### Do some publishers have favourite genres?

In [0]:
publisher_genre_data = data.groupBy('publisher', 'genre').count()
publisher_owner_data = data.groupBy('publisher').mean('owners')
publisher_data = publisher_genre_data.join(publisher_owner_data, 'publisher', 'left')
max_genre_publisher_data = data.groupBy('publisher', 'genre').count().groupBy('publisher').max('count')
favourite_publisher_genre_data = publisher_data.join(max_genre_publisher_data, 'publisher', 'left')


In [0]:
display(
    favourite_publisher_genre_data.filter(F.col('count') == F.col('max(count)')).orderBy(['avg(owners)', 'count'], ascending=False).limit(20)
)

publisher,genre,count,avg(owners),max(count)
Valve,Action,346,6.3068692196531795E7,346
Amazon Games,Action,20,4.6411764705882356E7,20
"KRAFTON, Inc.",Action,23,4.52279203539823E7,23
Wallpaper Engine Team,Animation & Modeling,23,3.5E7,23
Wallpaper Engine Team,Indie,23,3.5E7,23
Wallpaper Engine Team,Casual,23,3.5E7,23
Wallpaper Engine Team,Design & Illustration,23,3.5E7,23
Wallpaper Engine Team,Photo Editing,23,3.5E7,23
Wallpaper Engine Team,Utilities,23,3.5E7,23
"FromSoftware Inc., Bandai Namco Entertainment",RPG,14,3.5E7,14


t looks like action games are the genre that the biggest publishers like to publish the most, although we'd have to explore or data further if we wanted to be able to say that they publish those games significantly more than any other game. Note that we often have duplicate entires of publishers, this is due many of these publishers having published an even amount of games of multiple genres (don't forget games can have multiple genres!).

#### What are the most lucrative genres?

In [0]:
display(data.groupBy('genre').mean('revenue').orderBy('avg(revenue)', ascending=False).limit(10),
        data.groupBy('genre').mean('revenue').orderBy('avg(revenue)', ascending=True).limit(10))

genre,avg(revenue)
RPG,1.9517454411764707E7
Animation & Modeling,1.7885429281767957E7
Strategy,1.4149998035328045E7
Massively Multiplayer,1.2178633122553447E7
RPG,8339238.322788129
Simulation,7974726.925440393
Adventure,7938345.849042923
Photo Editing,7902480.968468469
Action,7289354.654875387
Racing,6537795.6701599555


MMOs and RPGs are unsurprisingly some of the most lucrative games out there, thanks to their service-based model.

# Platform analysis

#### Are most games available on Windows/Mac/Linux?

In [0]:
platforms = ['windows', 'linux', 'mac']

for platform in platforms:
  print(platform, data.filter(F.col(platform) == True).count()/data.count()*100, '%')

windows 99.98016013359994 %
linux 19.60937890866933 %
mac 29.25552195238087 %


As expected, almost all games are supported by Windows given it is the defacto consumer platform. Linux and Mac both have a similarly low level of support at 19% and 29% respectively, although this does not take into account the Proton emulator for Linux that makes a majority of Steam's library available on the platform.

#### Do certain genres tend to be preferentially available on certain platforms?

In [0]:
games_per_genre_data = data.groupBy('genre').agg(F.count('*').alias('genre_amount'))
windows_per_genre_data = data.filter(F.col('windows') == True).groupBy('genre').agg(F.count('*').alias('windows_amount'))
linux_per_genre_data = data.filter(F.col('linux') == True).groupBy('genre').agg(F.count('*').alias('linux_amount'))
mac_per_genre_data = data.filter(F.col('mac') == True).groupBy('genre').agg(F.count('*').alias('mac_amount'))

platform_genre_data = games_per_genre_data.join(windows_per_genre_data, 'genre', 'left') \
    .join(linux_per_genre_data, 'genre', 'left') \
    .join(mac_per_genre_data, 'genre', 'left')

platform_genre_data = platform_genre_data.withColumn('windows', F.round(F.col('windows_amount') / F.col('genre_amount'), 2)) \
    .withColumn('linux', F.round(F.col('linux_amount') / F.col('genre_amount'), 2)) \
    .withColumn('mac', F.round(F.col('mac_amount') / F.col('genre_amount'), 2))

In [0]:
display(platform_genre_data.orderBy('genre_amount', ascending=False).limit(20))

genre,genre_amount,windows_amount,linux_amount,mac_amount,windows,linux,mac
Indie,124887,124877,26874,38106,1.0,0.22,0.31
Action,87070,87058,16766,21692,1.0,0.19,0.25
Casual,47249,47246,9882,15364,1.0,0.21,0.33
Adventure,41376,41375,7592,9968,1.0,0.18,0.24
Casual,39712,39711,6930,10723,1.0,0.17,0.27
Adventure,39032,39023,7335,13767,1.0,0.19,0.35
Strategy,39010,39003,9197,14389,1.0,0.24,0.37
Simulation,37902,37895,8072,11808,1.0,0.21,0.31
RPG,28607,28606,5311,8119,1.0,0.19,0.28
Early Access,16480,16480,2346,3131,1.0,0.14,0.19


# Addtional Analysis

#### Do better rated games sell better?

In [0]:
data_rating = data.withColumn('rating_category', F.when(F.col('rating') > 0.95, 'Overwhelmingly Positive')
                                    .when((F.col('rating') > 0.80) & (F.col('rating') < 0.95), 'Very Positive')
                                    .when((F.col('rating') > 0.70) & (F.col('rating') < 0.80), 'Positive')
                                    .when((F.col('rating') > 0.40) & (F.col('rating') < 0.70), 'Mixed')
                                    .when((F.col('rating') > 0.20) & (F.col('rating') < 0.40), 'Mostly Negative')
                                    .when((F.col('rating') > 0.00) & (F.col('rating') < 0.20), 'Very Negative')
                                    )

In [0]:
display(
    data_rating.groupBy('rating_category').mean('owners', 'rating').orderBy('avg(rating)', ascending=False).drop('avg(rating)').withColumn('average_owners', F.round('avg(owners)', 0)).drop('avg(owners)').limit(10)
)

rating_category,average_owners
Overwhelmingly Positive,351933.0
Very Positive,626288.0
Positive,239794.0
Mixed,186960.0
null,15486.0
Mostly Negative,47728.0
Very Negative,19659.0


As expected, we see that the games with the best reviews often have more sales, with a very steep drop off once a game is considered "Mostly Negative" by Steam. We also see that "Overwhelmingly Positive" actually have lower ownership, but this could be due to games with very low review counts being considered.

In [0]:
display(
    data_rating.filter((F.col('positive') + F.col('negative')) > 1000).groupBy('rating_category').mean('owners', 'rating').orderBy('avg(rating)', ascending=False).drop('avg(rating)').withColumn('average_owners', F.round('avg(owners)', 0)).drop('avg(owners)').limit(10)
)

rating_category,average_owners
Overwhelmingly Positive,2336926.0
Very Positive,1958077.0
Positive,1065329.0
Mixed,1488003.0
Mostly Negative,1193608.0
Very Negative,176071.0



When we only look at games with significant amounts of reviews, the distribution becomes a lot more regular. Average owners steadily increases with ratings (with a small exception for mixed reviews that are slightly lower than expected, and very negative reviews which have very few players).

#### Do games that support Mac also support Linux? And vice versa?

In [0]:
linux_data = data.filter(F.col('linux') == True)
mac_data = data.filter(F.col('mac') == True)

In [0]:
print(
    'Percent of games that support Mac or Linux: ',
    data.filter((F.col('mac') == True) | (F.col('linux') == True)).count() / data.count() * 100,
    '\n',
    'Percent of Linux games that support Mac: ',
    linux_data.filter(F.col('mac') == True).count() / linux_data.count() * 100,
    '\n',
    'Percent of Mac games that support Linux: ',
    mac_data.filter(F.col('linux') == True).count() / mac_data.count() * 100
)

Percent of games that support Mac or Linux:  32.097798464911904 
 Percent of Linux games that support Mac:  85.50552505630631 
 Percent of Mac games that support Linux:  57.31260725216274



It does indeed look like games that support Mac or Linux support the other secondary platform more often. It makes sense that studios with the ressources and will to support multiple platforms would go through the effort of supporting both, rather than only one. We do note that Linux games are generally more cross-compatible, could this be due to the open-source nature of the platform and mindset of the developers that create for it?

#### How has the price of games evolved over the years?

In [0]:
display(
    data.groupBy(F.year('release_date')).mean('price').orderBy(F.year('release_date'), ascending=False).limit(100)
)

year(release_date),avg(price)
2022,11.043988809942753
2021,10.001366648445044
2020,8.632283256255551
2019,7.9693333955574275
2018,7.822365082973651
2017,7.686865315626159
2016,8.187749518300162
2015,8.379348498324983
2014,10.329954939098403
2013,11.524577234371861


#### Are certain genres more popular during certain seasons?

In [0]:
data_seasons = data.withColumn('season', F.when(F.month('release_date') < 3, 0)
                               .when(F.month('release_date') < 6, 1)
                               .when(F.month('release_date') < 9, 2)
                               .when(F.month('release_date') < 12, 3))

In [0]:
top_genres = data.groupBy('genre').count().orderBy('count', ascending=False).limit(10).rdd.map(lambda x: x[0]).collect()

In [0]:
display(data_seasons.filter((F.col('positive') + F.col('negative')) > 1000).groupBy('season').count().orderBy('season'))

season,count
null,9033
0,13686
1,27464
2,25951
3,30175


Games are released seasonally, starting with a relatively low quantity in spring that rapidly doubles and stays elevated for the rest of the year. We can suppose that this is all due to publishers trying to maximise their sales during the holiday season, when they know consumers will pay full price for games.

In [0]:
display(
  data_seasons.filter((F.col('positive') + F.col('negative')) > 1000).filter(F.col('genre').isin(top_genres)).groupBy('genre', 'season').count().orderBy(['genre', 'season'], ascending=False)
)

genre,season,count
Casual,3,965
Casual,2,942
Casual,1,867
Casual,0,476
Casual,null,443
Adventure,3,1917
Adventure,2,1589
Adventure,1,1807
Adventure,0,997
Adventure,null,470


The pattern we noticed earlier does indeed apply to all genres (or at least all the most popular ones).